In [19]:
import duckdb
import os
from dotenv import load_dotenv, find_dotenv
from deltalake import DeltaTable, write_deltalake

load_dotenv(find_dotenv())

con = duckdb.connect()

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_MINIO")
AWS_SECRET_KEY_ID = os.getenv("AWS_SECRET_KEY_MINIO")
HOST_MINIO = os.getenv("HOST_MINIO")
PORT_MINIO = os.getenv("PORT_MINIO")

con.sql(f""" CREATE SECRET secret1 (
            TYPE S3,
            KEY_ID '{AWS_ACCESS_KEY_ID}',
            SECRET '{AWS_SECRET_KEY_ID}',
            REGION 'us-east-1',
            ENDPOINT '{HOST_MINIO}:{PORT_MINIO}',
            URL_STYLE 'path',
            USE_SSL 'false'

        );
    """)

storage_options = {
    "AWS_ACCESS_KEY_ID": f"{AWS_ACCESS_KEY_ID}",
    "AWS_SECRET_ACCESS_KEY": f"{AWS_SECRET_KEY_ID}",
    "AWS_ENDPOINT_URL": f"http://{HOST_MINIO}:{PORT_MINIO}",
    "AWS_allow_http": "true",
    "AWS_REGION": "us-east-1",
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true",
}

In [20]:
path_silver = "s3://silver/uff/projeto_comex"
path_gold = "s3://gold/uff/projeto_comex"

In [21]:
# con.sql(f"""
#     SELECT
#     *
#     FROM delta_scan('{path_gold}/IMP')
# """)

In [ ]:
fat_imp = con.sql(f"""
        SELECT  ANO,
                ANO *100 + MES AS COD_DATA_MES,
                FK_NCM,
                FK_PAIS,
                FK_SIGA_ESTADO,
                QUANTIDADE,
                KG_LIQUIDO,
                VALOR_MERCADORIA,
                VALOR_FRETE,
                VALOR_SEGURO 
        FROM delta_scan('{path_silver}/IMP')
        WHERE CAST(ANOS*100 + MES AS INT) >
        (SELECT MAX(CAST(COD_DATA_MES AS INT)) FROM delta_scan('{path_gold}/FAT_IMP'))
                  """).to_arrow_table()

In [ ]:
if len(fat_imp) > 0:
        write_deltalake(
                f'{path_gold}/FAT_IMP',
                fat_imp,
                mode='append',
                storage_options=storage_options,
                partition_by= ['ANO']
        )

In [8]:
con.close()